<a href="https://colab.research.google.com/github/Sarath-kumar-0-3/231801156-AI23431-Web-Technology-and-Mobile-Application-/blob/main/jokernew03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os

zip_path = "/content/Task 3 - Onomastic Wordplay-20250518T124320Z-1-001.zip"
extract_dir = "/content/task3_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Show extracted files
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        print(os.path.join(root, file))


/content/task3_data/Task 3 - Onomastic Wordplay/joker_onomastic_2025_test.json
/content/task3_data/Task 3 - Onomastic Wordplay/joker_onomastic_2025_train.json


In [2]:
import json
import pandas as pd

# Paths to your train and test files
train_path = "/content/task3_data/Task 3 - Onomastic Wordplay/joker_onomastic_2025_train.json"
test_path = "/content/task3_data/Task 3 - Onomastic Wordplay/joker_onomastic_2025_test.json"

# Load the JSON files
with open(train_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open(test_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Convert to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Show some examples
print("Training Data Sample:")
display(train_df.head())

print("Test Data Sample:")
display(test_df.head())


Training Data Sample:


,id,en,description,fr
0,en_1,Asterix,Asterix is the small but clever hero of the As...,Astérix
1,en_2,Obelix,Obélix is one of the main characters in the As...,Obélix
2,en_3,Dogmatix,"Dogmatix (Idéfix in French) is a small, white ...",Idéfix
3,en_4,Getafix,"Getafix, known as Panoramix in French, is the ...",Panoramix
4,en_5,Vitalstatistix,Vitalstatistix is the fearless and somewhat po...,Abraracourcix


Test Data Sample:


,id,en,description
0,en_1,Asterix,Asterix is the small but clever hero of the As...
1,en_2,Obelix,Obélix is one of the main characters in the As...
2,en_3,Dogmatix,"Dogmatix (Idéfix in French) is a small, white ..."
3,en_4,Getafix,"Getafix, known as Panoramix in French, is the ..."
4,en_5,Vitalstatistix,Vitalstatistix is the fearless and somewhat po...


In [3]:
!pip install -q transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.6 MB/s eta 0:00:00


In [4]:
from transformers import T5Tokenizer
from datasets import Dataset

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

# Preprocess function
def preprocess_function(example):
    input_text = f"Translate with context: {example['en']} ||| {example['description']}"
    target_text = example["fr"]
    model_inputs = tokenizer(input_text, max_length=128, padding="max_length", truncation=True)
    labels = tokenizer(target_text, max_length=32, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Convert train DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)

# Tokenize
tokenized_train = train_dataset.map(preprocess_function, remove_columns=train_dataset.column_names)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/353 [00:00<?, ? examples/s]

In [5]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

training_args = TrainingArguments(
    output_dir="./flan_t5_onomastic",
    per_device_train_batch_size=4,
    num_train_epochs=8,
    logging_steps=50,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
)

trainer.train()


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<ipython-input-5-2d0bc83d7d1a>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sarathsara2733 (sarathsara2733-anna-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,17.437300
100,3.861900
150,1.567900
200,0.820300
250,0.764800
300,0.673000
350,0.609600
400,0.595200
450,0.573700
500,0.544100


TrainOutput(global_step=712, training_loss=2.0819765115052125, metrics={'train_runtime': 393.1276, 'train_samples_per_second': 7.183, 'train_steps_per_second': 1.811, 'total_flos': 483438690828288.0, 'train_loss': 2.0819765115052125, 'epoch': 8.0})

In [6]:
def generate_translation(row):
    input_text = f"Translate with context: {row['en']} ||| {row['description']}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(model.device)
    output = model.generate(**inputs, max_new_tokens=32)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Apply inference
test_df['fr_generated'] = test_df.apply(generate_translation, axis=1)
test_df[['en', 'fr_generated']]


,en,fr_generated
0,Asterix,Asterix
1,Obelix,Obélix
2,Dogmatix,Idéfix
3,Getafix,Panoramix
4,Vitalstatistix,Vitalstatistix
...,...,...
2691,Nightwights,Nights
2692,The Peppered Tyrant,The Auric Godshawk
2693,Welcome Break,Welcome Break
2694,Large Blunt Object,Large bouillon d'or


In [7]:
import json

# Replace 'yourteam' with your actual team name
run_id = "Cryptix_task_3_flanT5"

submission = [
    {
        "run_id": run_id,
        "manual": 0,
        "id_en": row["id"],
        "en": row["en"],
        "fr": row["fr_generated"]
    }
    for _, row in test_df.iterrows()
]

# Save as JSON
with open("joker_task3_submission.json", "w", encoding="utf-8") as f:
    json.dump(submission, f, ensure_ascii=False, indent=2)

print("✅ Submission file created: joker_task3_submission.json")


✅ Submission file created: joker_task3_submission.json


In [8]:
import random
print(json.dumps(random.choice(submission), indent=2, ensure_ascii=False))


{
  "run_id": "Cryptix_task_3_flanT5",
  "manual": 0,
  "id_en": "en_103",
  "en": "Bravura",
  "fr": "Bravura"
}


In [ ]:
!zip -j task3_submission.zip "/content/joker_task3_submission.json"


  adding: joker_task3_submission.json (deflated 89%)


In [ ]:
from google.colab import files
files.download("task3_submission.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
from sklearn.model_selection import train_test_split

train_split, val_split = train_test_split(train_df, test_size=0.2, random_state=42)


In [10]:
def generate_translation(row):
    input_text = f"Translate with context: {row['en']} ||| {row['description']}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(model.device)
    output = model.generate(**inputs, max_new_tokens=32)
    return tokenizer.decode(output[0], skip_special_tokens=True)

val_split["fr_predicted"] = val_split.apply(generate_translation, axis=1)


In [11]:
!pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

smoothie = SmoothingFunction().method4


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [12]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

smoothie = SmoothingFunction().method4

def evaluate_scores(reference, prediction):
    # Tokenize both reference and prediction
    ref_tokens = word_tokenize(reference)
    pred_tokens = word_tokenize(prediction)

    # BLEU expects tokenized words
    bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothie)

    # METEOR also expects tokenized input
    meteor = meteor_score([ref_tokens], pred_tokens)

    return bleu, meteor


In [13]:
def generate_translation(row):
    input_text = f"Translate with context: {row['en']} ||| {row['description']}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(model.device)
    output = model.generate(**inputs, max_new_tokens=32)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Apply to each row in val_split
val_split["fr_generated"] = val_split.apply(generate_translation, axis=1)


In [14]:
import nltk
nltk.download('punkt')       # for word tokenization
nltk.download('wordnet')     # for METEOR


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

smoothie = SmoothingFunction().method4

def simple_tokenize(text):
    return text.strip().lower().split()

def evaluate_scores(reference, prediction):
    ref_tokens = simple_tokenize(reference)
    pred_tokens = simple_tokenize(prediction)

    bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothie)
    meteor = meteor_score([ref_tokens], pred_tokens)  # tokenized input
    return bleu, meteor


In [17]:
bleu_scores = []
meteor_scores = []

for _, row in val_split.iterrows():
    bleu, meteor = evaluate_scores(row["fr"], row["fr_generated"])
    bleu_scores.append(bleu)
    meteor_scores.append(meteor)

print(f"\nAverage BLEU: {sum(bleu_scores)/len(bleu_scores):.4f}")
print(f"Average METEOR: {sum(meteor_scores)/len(meteor_scores):.4f}")



Average BLEU: 0.1516
Average METEOR: 0.1106
